# Coordinate Conversions
This notebook tutorial shows how to convert between coordinate systems using a sample trajectory retrieved from SSCWeb.

In [1]:
# Import satellite flythrough coordinate conversion code for more info on the coordinate systems.
from kamodo_ccmc.flythrough.utils import ConvertCoord
help(ConvertCoord)

Help on function ConvertCoord in module kamodo_ccmc.flythrough.utils:

ConvertCoord(inTime, c1, c2, c3, inCoord, inType, outCoord, outType, verbose=False)
    This function uses spacepy and astropy to convert time and position arrays
    from one coordinate system to another. It will correct obvious errors in
    the return units, but may not catch all incorrect values.
    
    INPUTS:
    inTime array:  time in UTC timestamp
    c1 array:  x (in R_earth)*, lon (in deg)
    c2 array:  y (in R_earth)*, lat (in deg)
    c3 array:  z (in R_earth)*, alt (in km), radius (in R_earth)
    inCoord string: case-sensitive string from list:
        'GDZ', 'GEO', 'GSM', 'GSE', 'SM', 'GEI', 'MAG', 'SPH', 'RLL'
        (SpacePy coordinates)
        'teme', 'icrs', 'fk5', 'fk4', 'itrs', 'galactic', 'galactocentric',
        'cirs', 'tete', 'precessedgeocentric', 'geocentricmeanecliptic',
        'geocentrictrueecliptic', 'hcrs', 'barycentricmeanecliptic',
        'heliocentricmeanecliptic', 'barycen

C:\Users\rringuet\Anaconda3\envs\Kamodo_Jan2023\lib\site-packages\spacepy\time.py:2367: UserWarning: Leapseconds may be out of date. Use spacepy.toolbox.update(leapsecs=True)
  warnings.warn('Leapseconds may be out of date.'


## Retrieve a real satellite trajectory as input

In [2]:
# Typical coordinates possible through SSCWeb are GEO, GSE, SM, and GSM (all cartesian and in R_E).
from kamodo_ccmc.flythrough import SatelliteFlythrough as SF
from datetime import datetime, timezone
start = datetime(2015, 3, 18, 6, 26, 40).replace(tzinfo=timezone.utc)
end = datetime(2015, 3, 18, 12, 11, 40).replace(tzinfo=timezone.utc)
traj_dict, coord_type = SF.SatelliteTrajectory('grace1', start.timestamp(), end.timestamp(), coord_type='GEO')
# Show the range of each coordinate. 
print('Coordinate system:', coord_type)
print('sat_time = utc timestamp:', traj_dict['sat_time'].min(), traj_dict['sat_time'].max())
print(datetime.utcfromtimestamp(traj_dict['sat_time'].min()), datetime.utcfromtimestamp(traj_dict['sat_time'].max()))
print('c1 = x(R_E):', traj_dict['c1'].min(), traj_dict['c1'].max())
print('c2 = y(R_E):', traj_dict['c2'].min(), traj_dict['c1'].max())
print('c3 = z(R_E):', traj_dict['c3'].min(), traj_dict['c1'].max())

Attribute/Key names of return dictionary: dict_keys(['sat_time', 'c1', 'c2', 'c3'])
Coordinate system: GEO-car
sat_time = utc timestamp: 1426660020.0 1426680660.0
2015-03-18 06:27:00 2015-03-18 12:11:00
c1 = x(R_E): -0.9270195119 1.0114874276
c2 = y(R_E): -1.0605838293 1.0114874276
c3 = z(R_E): -1.0639983632 1.0114874276


## Convert to a spherical coordinate system in SpacePy

In [3]:
# The syntax to convert:
c1, c2, c3, units = ConvertCoord(traj_dict['sat_time'], traj_dict['c1'], traj_dict['c2'], traj_dict['c3'], 
                                 *coord_type.split('-'), 'SM', 'sph')
# Show the range of each coordinate. Note the time does not change.
print('Coordinate system: SM-sph')
print('c1 = longitude(deg):', c1.min(), c1.max())
print('c2 = latitude(deg):', c2.min(), c2.max())
print('c3 = radius(R_E):', c3.min(), c3.max())
print('units (c1, c2, c3):',units)

Coordinate system: SM-sph
c1 = longitude(deg): -175.82629680633556 150.11391934929247
c2 = latitude(deg): -88.63688132453802 88.71473929655991
c3 = radius(R_E): 1.0599501466186172 1.0644604465687955
units (c1, c2, c3): ['deg', 'deg', 'R_E']


## Convert to a cartesian coordinate system in AstroPy

In [4]:
# The syntax to convert:
c1, c2, c3, units = ConvertCoord(traj_dict['sat_time'], traj_dict['c1'], traj_dict['c2'], traj_dict['c3'], 
                                 *coord_type.split('-'), 'teme', 'car')
# Show the range of each coordinate. Note the time does not change
print('Coordinate system: teme-car')
print('c1 = x(R_E):', c1.min(), c1.max())
print('c2 = y(R_E):', c2.min(), c2.max())
print('c3 = z(R_E):', c3.min(), c3.max())
print('units (c1, c2, c3):',units)

Coordinate system: teme-car
c1 = x(R_E): -0.2046378793907867 0.20505787042036003
c2 = y(R_E): -1.0421557059743505 1.0437086971696456
c3 = z(R_E): -1.0640045699238057 1.0598887612232442
units (c1, c2, c3): ['R_E', 'R_E', 'R_E']
